In [1]:
import glob
import pandas as pd
catfs = glob.glob("train/train/cat.*.jpg")
catt = [0] * len(catfs)
dogfs = glob.glob("train/train/dog.*.jpg")
dogt = [1] * len(dogfs)
df = pd.DataFrame({
    "path":catfs + dogfs,
    "ans":catt + dogt
})
df

,path,ans
0,train/train\cat.0.jpg,0
1,train/train\cat.1.jpg,0
2,train/train\cat.10.jpg,0
3,train/train\cat.100.jpg,0
4,train/train\cat.1000.jpg,0
...,...,...
24995,train/train\dog.9995.jpg,1
24996,train/train\dog.9996.jpg,1
24997,train/train\dog.9997.jpg,1
24998,train/train\dog.9998.jpg,1


In [2]:
from keras.applications.vgg16 import VGG16
vgg = VGG16(include_top=False, input_shape=(224, 224, 3))
vgg.summary()

Using TensorFlow backend.


Instructions for updating:
If using Keras pass *_constraint arguments to layers.

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_____________________________________________

In [3]:
vgg.layers

In [4]:
from keras.models import Model
from keras.layers import Dense, Dropout, Flatten
# trainable一定要在compile前
for l in vgg.layers:
    l.trainable = False
x = Flatten()(vgg.output)
x = Dense(512, activation="relu")(x)
x = Dropout(0.25)(x)
out = Dense(2, activation="softmax")(x)
model = Model(inputs=vgg.input, outputs=out)
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0   

In [5]:
model.compile(loss="sparse_categorical_crossentropy",
              optimizer="adam",
              metrics=["accuracy"])

In [6]:
# https://github.com/keras-team/keras-applications/blob/master/keras_applications/imagenet_utils.py
import numpy as np
from keras.preprocessing.image import load_img
from keras.applications.vgg16 import preprocess_input
p = df.iloc[0]["path"]
img = load_img(p, target_size=(224, 224)).convert("RGB")
img_np = np.array(img)
img_pre = preprocess_input(img_np)
img_pre

array([[[ -16.939003 ,   47.221    ,   79.32     ],
        [ -13.939003 ,   50.221    ,   82.32     ],
        [ -10.939003 ,   53.221    ,   85.32     ],
        ...,
        [  15.060997 ,   86.221    ,  121.32     ],
        [  18.060997 ,   85.221    ,  119.32     ],
        [  17.060997 ,   83.221    ,  115.32     ]],

       [[ -16.939003 ,   47.221    ,   79.32     ],
        [ -13.939003 ,   50.221    ,   82.32     ],
        [ -10.939003 ,   53.221    ,   85.32     ],
        ...,
        [  16.060997 ,   88.221    ,  121.32     ],
        [  19.060997 ,   87.221    ,  118.32     ],
        [  18.060997 ,   84.221    ,  116.32     ]],

       [[ -16.939003 ,   47.221    ,   79.32     ],
        [ -13.939003 ,   50.221    ,   82.32     ],
        [ -10.939003 ,   53.221    ,   85.32     ],
        ...,
        [  18.060997 ,   87.221    ,  121.32     ],
        [  20.060997 ,   88.221    ,  119.32     ],
        [  19.060997 ,   85.221    ,  117.32     ]],

       ...,

      

In [7]:
# 試試看numpy的randint
ori = np.random.randint(0, 10, 5)
new = list(map(lambda x:x**2, ori))
print(ori)
print(new)

[3 2 9 8 7]
[9, 4, 81, 64, 49]


In [8]:
def preprocess(path):
    img = load_img(path, target_size=(224, 224)).convert("RGB")
    img_np = np.array(img)
    img_pre = preprocess_input(img_np)
    return img_pre

def get_images(paths, targets, batch=20):
    idx = np.random.randint(0, len(paths), batch)
    ps = paths[idx]
    xs = np.array(list(map(preprocess, ps)))
    ys = targets[idx]
    return (ps, xs, ys)

In [9]:
from sklearn.model_selection import train_test_split
x = np.array(df["path"])
y = np.array(df["ans"])
x_train, x_test, y_train, y_test = train_test_split(x, y,
                                                    test_size=0.1)
ps, xs, ys = get_images(x_train, y_train)
print(ys)

[0 0 1 1 1 1 0 0 1 0 1 1 1 0 1 1 1 1 1 1]


In [10]:
for i in range(10):
    print("-" * 15, "Times:" , i, "-" * 15)
    _, xs, ys = get_images(x_train, y_train)
    train_loss = model.train_on_batch(xs, ys)
    print("[Train]:", train_loss)
    _, xs, ys = get_images(x_test, y_test)
    val_loss = model.test_on_batch(xs, ys)
    print("[Validate]:", val_loss)

--------------- Times: 0 ---------------

[Train]: [16.924963, 0.3]
[Validate]: [58.342083, 0.5]
--------------- Times: 1 ---------------
[Train]: [38.182503, 0.65]
[Validate]: [41.715057, 0.7]
--------------- Times: 2 ---------------
[Train]: [40.860863, 0.65]
[Validate]: [3.7635074, 0.95]
--------------- Times: 3 ---------------
[Train]: [2.3050818, 0.95]
[Validate]: [1.2098279e-05, 1.0]
--------------- Times: 4 ---------------
[Train]: [9.796427, 0.85]
[Validate]: [1.1368264, 0.95]
--------------- Times: 5 ---------------
[Train]: [24.77062, 0.8]
[Validate]: [2.7138124, 0.95]
--------------- Times: 6 ---------------
[Train]: [4.07841, 0.95]
[Validate]: [3.100139, 0.9]
--------------- Times: 7 ---------------
[Train]: [20.473866, 0.8]
[Validate]: [1.9816437, 0.95]
--------------- Times: 8 ---------------
[Train]: [0.042163987, 0.95]
[Validate]: [12.813504, 0.85]
--------------- Times: 9 ---------------
[Train]: [3.0081344, 0.95]
[Validate]: [0.0, 1.0]


In [11]:
_, xs, ys = get_images(x_test, y_test, 100)
model.evaluate(xs, ys)

100/100 [==============================] - 12s 120ms/step


[8.447997741699218, 0.9399999976158142]

In [12]:
model.predict(xs)

array([[1.0000000e+00, 2.1002033e-09, 0.0000000e+00],
       [0.0000000e+00, 1.0000000e+00, 0.0000000e+00],
       [0.0000000e+00, 1.0000000e+00, 0.0000000e+00],
       [0.0000000e+00, 1.0000000e+00, 0.0000000e+00],
       [0.0000000e+00, 1.0000000e+00, 0.0000000e+00],
       [0.0000000e+00, 1.0000000e+00, 0.0000000e+00],
       [0.0000000e+00, 1.0000000e+00, 0.0000000e+00],
       [2.9366308e-07, 9.9999976e-01, 0.0000000e+00],
       [0.0000000e+00, 1.0000000e+00, 0.0000000e+00],
       [1.0000000e+00, 0.0000000e+00, 0.0000000e+00],
       [9.9999988e-01, 1.0988133e-07, 0.0000000e+00],
       [1.0000000e+00, 0.0000000e+00, 0.0000000e+00],
       [1.0000000e+00, 0.0000000e+00, 0.0000000e+00],
       [0.0000000e+00, 1.0000000e+00, 0.0000000e+00],
       [0.0000000e+00, 1.0000000e+00, 0.0000000e+00],
       [1.0000000e+00, 0.0000000e+00, 0.0000000e+00],
       [1.0000000e+00, 0.0000000e+00, 0.0000000e+00],
       [1.0000000e+00, 0.0000000e+00, 0.0000000e+00],
       [1.0000000e+00, 0.000